# Hazard assessment for Infrastructures using Euro-Cordex datasets
## Calculation of the Return levels for the return periods of 10, 20, 30, 50, 100, 150 years

- See our [how to use risk workflows](https://handbook.climaax.eu/notebooks/workflows_how_to.html) page for information on how to run this notebook.

## Hazard assessment methodology
We utilized outputs from 14 models within the EURO-CORDEX framework to evaluate hazards affecting infrastructure, in this notebook we used the total daily precipitation as an indicator of the hazzard. Our analysis included three Representative Concentration Pathways (RCPs): RCP2.6, RCP4.5, and RCP8.5. To structure the future projections, we divided the RCP scenarios into three distinct periods: 2021–2050, 2041–2070, and 2071–2100. Additionally, we used the historical period (1981–2010) as a baseline for comparison.

For each period and model, the variation of the indicator "percentile associated to a return periods of 10, 20, 30, 50, 100, 150 years" is calculated by comparing the future scenarios with the historical data. Finally, once the results for each model are obtained, the ensemble mean is computed. The results of these computations will be visualized in the notebook *07_cordex_returnLevelsPrecip_plots.ipynb* to assess potential future hazards.


### Select area of interest
Before downloading the data, we will define the coordinates of the area of interest, for this workflow we selected the Italy region. Based on the shapefile of the country we will be able to clip the datasets for further processing, and display hazard and damage maps for this area.

### Load libraries

In [ ]:
import cftime
import datetime
import os
import xarray as xr
import numpy as np
from scipy.stats import gumbel_r

### Create the directory structure

In [ ]:
# Define paths and folders
data_path = '/work/cmcc/dg07124/climax/data/cordex/precip'
folders = ['historical', 'rcp26', 'rcp45', 'rcp85']
avg_models_path = '/work/cmcc/dg07124/climax/indicators/cordex/avg_modelsPrecip'
return_levels_path = '/work/cmcc/dg07124/climax/indicators/cordex/returnLevelsPrecip'

In [ ]:
# Ensure output directories exist
os.makedirs(avg_models_path, exist_ok=True)
os.makedirs(return_levels_path, exist_ok=True)

# Define time ranges for processing
rcp_time_ranges = [('2021', '2050'), ('2041', '2070'), ('2071', '2100')]
historical_time_range = [('1981', '2010')]

# Define the return periods and their exceedance probabilities
return_periods = np.array([10, 20, 30, 50, 100, 150])
exceedance_probs = 1 - (1 / return_periods)

In [ ]:

# Process each folder
for folder in folders:
    folder_path = os.path.join(data_path, folder)
    
    # Load all NetCDF files in the folder and extract 'pr' variable to average
    datasets = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.nc'):
            ds = xr.open_dataset(os.path.join(folder_path, filename))

            # Check if time is already in a CFTime format
            if isinstance(ds.indexes['time'][0], cftime.DatetimeNoLeap):
                # Already in the desired 'noleap' format, no conversion needed
                pass
            elif isinstance(ds.indexes['time'][0], cftime._cftime.DatetimeAllLeap):
                # Convert from 'all_leap' to 'noleap' by recreating the time index
                start_date = ds.indexes['time'][0].strftime('%Y-%m-%d')
                ds['time'] = xr.cftime_range(
                    start=start_date,
                    periods=len(ds['time']),
                    freq='D',
                    calendar='noleap'
                )
            else:
                # Assume 'time' is numpy.datetime64; convert to 'noleap' CFTimeIndex
                start_date = str(ds['time'].values[0].astype('M8[D]'))  # Convert to ISO string
                ds['time'] = xr.cftime_range(
                    start=start_date,
                    periods=len(ds['time']),
                    freq='D',
                    calendar='noleap'
                )


            # Convert 'pr' to mm/day by multiplying by 86400
            pr_mm_day = ds['pr'] * 86400
            pr_mm_day.attrs['units'] = 'mm/day'  # Update units attribute
            datasets.append(pr_mm_day)              
            # datasets.append(ds['pr'])  # Assuming 'pr' is the precipitation variable name

    # Stack datasets along a new 'model' dimension and compute the mean across models
    combined_ds = xr.concat(datasets, dim='model').mean(dim='model')

    # Ensure latitude and longitude coordinates are retained
    if 'lat' in datasets[0].coords and 'lon' in datasets[0].coords:
        combined_ds = combined_ds.assign_coords({
            'lat': datasets[0]['lat'],  # Retain the 2D latitude coordinates
            'lon': datasets[0]['lon']   # Retain the 2D longitude coordinates
        })

    # Save the averaged dataset
    avg_file_path = os.path.join(avg_models_path, f"{folder}_avgPrecip.nc")
    combined_ds.to_netcdf(avg_file_path)
    print(f"Averaged NetCDF saved to: {avg_file_path}")


    # Determine the time ranges to use
    time_ranges = historical_time_range if folder == 'historical' else rcp_time_ranges


    # Process each specified time range
    for start_year, end_year in time_ranges:
        # Subset data to the specified time range
        ds_subset = combined_ds.sel(time=slice(start_year, end_year))
        
        # Calculate the annual maximum precipitation for each year
        annual_max_precip = ds_subset.resample(time='YE').max()
        
        return_levels_ds = xr.Dataset()
        
        if 'lat' in datasets[0].coords and 'lon' in datasets[0].coords:
            return_levels_ds = return_levels_ds.assign_coords({
                'lat': datasets[0]['lat'],  # Retain 2D latitude coordinates
                'lon': datasets[0]['lon']   # Retain 2D longitude coordinates
            })


        for y_coord in range(annual_max_precip.sizes['y']):
            #for x_coord in annual_max_precip['x']:
            for x_coord in range(annual_max_precip.sizes['x']):
                # Extract the annual maximum series for the current grid cell
                annual_max_values = annual_max_precip.sel(x=x_coord, y=y_coord).values
                annual_max_values = annual_max_values[~np.isnan(annual_max_values)]  # Remove NaNs
                
                if len(annual_max_values) > 0:
                    # Fit the Gumbel distribution to the annual maxima for this grid cell
                    loc, scale = gumbel_r.fit(annual_max_values)
                    
                    # Calculate return levels for each return period
                    return_levels = gumbel_r.ppf(exceedance_probs, loc, scale)
                    
                    # Store return levels in the dataset
                    for rp, rl in zip(return_periods, return_levels):
                        return_period_label = f"return_period_{rp}_y"
                        if return_period_label not in return_levels_ds:
                            return_levels_ds[return_period_label] = xr.full_like(annual_max_precip.isel(time=0), np.nan)
                        return_levels_ds[return_period_label][y_coord, x_coord] = rl
        
        # Save the return levels dataset for the current time range
        return_levels_file_path = os.path.join(
            return_levels_path, f"{folder}_return_levels_{start_year}_{end_year}.nc"
        )
        return_levels_ds.to_netcdf(return_levels_file_path)
        print(f"Return levels NetCDF saved to: {return_levels_file_path}")

## Anomalies calculation

In [ ]:
# Define paths
return_levels_path = '/work/cmcc/dg07124/climax/indicators/cordex/returnLevelsPrecip'
output_path = '/work/cmcc/dg07124/climax/indicators/cordex/returnLevels_anomalies'
os.makedirs(output_path, exist_ok=True)

# Define scenarios and time ranges
scenarios = ['rcp26', 'rcp45', 'rcp85']
time_ranges = [('2021', '2050'), ('2041', '2070'), ('2071', '2100')]
historical_file = os.path.join(return_levels_path, "historical_return_levels_1981_2010.nc")

# Load the historical return levels dataset
historical_ds = xr.open_dataset(historical_file)
historical_ds_var =historical_ds['return_period_10_y']

In [ ]:
# Ensure lat and lon are present
if 'lat' not in historical_ds.coords or 'lon' not in historical_ds.coords:
    raise ValueError("Historical dataset is missing 'lat' or 'lon' coordinates")

print(historical_ds['lat'].values)
print(historical_ds['lon'].values)

In [ ]:
# Loop through each scenario and time range
for scenario in scenarios:
    for start_year, end_year in time_ranges:
        # Construct file path for the scenario return levels dataset
        scenario_file = os.path.join(return_levels_path, f"{scenario}_return_levels_{start_year}_{end_year}.nc")
        if not os.path.exists(scenario_file):
            print(f"File not found: {scenario_file}")
            continue
        
        # Load the scenario return levels dataset
        scenario_ds = xr.open_dataset(scenario_file)
        
        # Calculate the anomaly (scenario - historical)
        anomaly_ds = scenario_ds - historical_ds
        anomaly_ds.attrs['description'] = f"Return level anomalies for {scenario} ({start_year}-{end_year}) relative to historical (1981-2010)"

        # Assign lat and lon correctly to anomaly_ds
        anomaly_ds = anomaly_ds.assign_coords({
            'lat': (('y', 'x'), historical_ds['lat'].values),  # Assign lat as 2D coordinates
            'lon': (('y', 'x'), historical_ds['lon'].values)   # Assign lon as 2D coordinates
        })


        # Calculate the percentage change ((scenario - historical) / historical) * 100
        percentage_change_ds = (anomaly_ds / historical_ds) * 100
        percentage_change_ds.attrs['description'] = f"Percentage change in return levels for {scenario} ({start_year}-{end_year}) relative to historical (1981-2010)"
        
        # Assign lat and lon correctly to percentage_change_ds
        percentage_change_ds = percentage_change_ds.assign_coords({
            'lat': (('y', 'x'), historical_ds['lat'].values),  # Assign lat as 2D coordinates
            'lon': (('y', 'x'), historical_ds['lon'].values)   # Assign lon as 2D coordinates
        })

        # Define file paths for saving
        anomaly_file = os.path.join(output_path, f"{scenario}_anomaly_return_levels_{start_year}_{end_year}.nc")
        percentage_change_file = os.path.join(output_path, f"{scenario}_percentage_change_return_levels_{start_year}_{end_year}.nc")
       
        # Save the anomaly and percentage change datasets
        anomaly_ds.to_netcdf(anomaly_file)
        percentage_change_ds.to_netcdf(percentage_change_file)
        
        print(f"Anomaly file saved to: {anomaly_file}")
        print(f"Percentage change file saved to: {percentage_change_file}")

## Contributors
- Giuseppe Giugliano (giuseppe.giugliano@cmcc.it)
- Carmela de Vivo (carmela.devivo@cmcc.it)
- Daniela Quintero (daniela.quintero@cmcc.it)